In [2]:
import re
import pandas as pd
import numpy as np
import json
import os


DATA_IN_PATH='./data_in/'
TRAIN_CLEAN_DATA='train_clean.csv'

train_data=pd.read_csv(DATA_IN_PATH+TRAIN_CLEAN_DATA, header=0,quoting=3)

reviews=list(train_data['review'])
sentiments=list(train_data['sentiment'])

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000)

X=vectorizer.fit_transform(reviews)

In [105]:
from sklearn.model_selection import train_test_split

RANDOM_SEED=42
TEST_SPLIT=0.2

y=np.array(sentiments)

X_train, X_eval, y_train, y_eval=train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [106]:
from sklearn.linear_model import LogisticRegression

lgs=LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [107]:
# 검증 데이터로 성능 측정
print("Accuracy: {:f}".format(lgs.score(X_eval, y_eval)))

Accuracy: 0.859800


In [108]:
TEST_CLEAN_DATA='test_clean.csv'

test_data=pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA, header=0, quoting=0)

print(test_data)

                                                  review        id
0      naturally film main themes mortality nostalgia...  12311_10
1      movie disaster within disaster film full great...    8348_2
2      movie kids saw tonight child loved one point k...    5828_4
3      afraid dark left impression several different ...    7186_2
4      accurate depiction small time mob life filmed ...   12128_7
...                                                  ...       ...
24995  sony pictures classics looking sony got rights...   2155_10
24996  always felt ms merkerson never gotten role fit...     59_10
24997  disappointed movie familiar case read mark fuh...    2531_1
24998  opening sequence filled black white shots remi...    7772_8
24999  great horror film people want vomit retching g...  11465_10

[25000 rows x 2 columns]


In [109]:
testDataVecs=vectorizer.transform(test_data['review'])

In [110]:
test_predicted=lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [111]:
DATA_OUT_PATH='./data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids=list(test_data['id'])
answer_dataset=pd.DataFrame({'id':ids, 'sentiment':test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv',index=False)